# An introduction to conditional inference trees in R

This is an interactive Jupyter notebook for the workshop [*An introduction to conditional inference trees in R*](https://github.com/MartinSchweinberger/TreesUBonn) offered at Rheinische Friedrich-Wilhelms-Universität Bonn by Martin Schweinberger.  

**Preparation and session set up**

We set up our session by activating the packages we need for this tutorial. 


In [ ]:
# load packages
library(openxlsx)
library(dplyr)
library(ggparty)
library(stringr)


Once you have initiated the session by executing the code shown above, you are good to go.

If you are using this notebook on your own computer and you have not already installed the R packages listed above, you need to install them. You can install them by replacing the `library` command with `install.packages` and putting the name of the package into quotation marks like this: `install.packages("dplyr")`. Then, you simply run this command and R will install the package you specified.



***

## Using your own data

While the tutorial uses data from the LADAL website, you can also use your own data. You can see below what you need to do to upload and use your own data.

The code chunk below allows you to upload two files from your own computer. To be able to load your own data, you need to click on the folder symbol to the left of the screen:

![Binder Folder Symbol](https://slcladal.github.io/images/binderfolder.JPG)


Then on the upload symbol.

![Binder Upload Symbol](https://slcladal.github.io/images/binderupload.JPG)

Next, upload the files you want to analyze and then the respective files names in the file argument of the scan function. When you then execute the code (like to code chunk below, you will upload your own data.


In [ ]:
mytable1 <- openxlsx::read.xlsx("testdata1.xlsx", sheet = 1)
# inspect
mytable1


**Keep in mind though that you need to adapt the names of the texts in the code chunks below so that the code below work on your own texts!**

***


# Conditional Inference Trees

Conditional Inference Trees (CITs) are very good at determining the *true* effect of a predictor, i.e. the effect of a predictor if all other effects are simultaneously considered. In contrast to CARTs, CITs use p-values to determine splits in the data. 

Below is a conditional inference tree which shows how and what factors contribute to the use of discourse *like*. In conditional inference trees predictors are only included if the predictor is significant (i.e. if these predictors are necessary). 


## Example 1: disourse like

In a first step, we load the data (in this case, the data is stored online and made available via the LADAL github repo).


In [ ]:
citdata <- read.delim("https://slcladal.github.io/data/treedata.txt", header = T, sep = "\t")
# inspect the data (head shows the first 6 lines)
head(citdata)


We can also inspect the structure of the data using the `str()` function as shown below.



In [ ]:
# inspect the data (str shows the structure of the data)
str(citdata)


We can see that all variables are character variables (indicated by the `chr` next to the variable name). However, CITs require factors (not character strings) and we thus need to convert the variables into factors (which we can do using `mutate_if(is.character, factor)`) 



In [ ]:
# convert character strings to factors
citdata <- citdata %>%
  dplyr::mutate_if(is.character, factor)
# inspect the data (str shows the structure of the data)
str(citdata)


Now we can implement the CIT. We start by defining a so-called control which allows us to model that we are running several test in one go. Setting a control in this case means that we lower the significance levels so that the overall significance level (of all test combined is still at $\alpha$ = .05 (this prevents errors in our analysis called *false positives*).  



In [ ]:
# apply bonferroni correction (1 minus alpha multiplied by n of predictors)
control = ctree_control(mincriterion = 1-(.05*ncol(citdata)-1))


**WARNING: we are actually not going to use the control here!** In a real analysis, you would add `, control = ctree_control(mincriterion = 1-(.05*ncol(citdata)-1))` after the data specification in the `ctree` function below but for this example, we are ignoring this (as the results would be pretty boring).

After setting the control, we implement the CIT by using the `ctree` function from the `partykit` package. This function requires the formula (the dependent variable ~ and the independent variables) and the specification of the data. Then, we plot the resulting CIT using the `plot` function.


In [ ]:
# create initial conditional inference tree model
citd.ctree <- partykit::ctree(LikeUser ~ Age + Gender + Status,
                    data = citdata)
plot(citd.ctree, gp = gpar(fontsize = 8)) # plot final ctree


We  can save the figure using the `ggsave` command as shown below.



In [ ]:
ggsave("tree1.png")



## Example 2: Prepositions

We now proceed with a different example that uses a numeric dependent variable. In this example, we want to see what factors impact the use of prepositions across time in historical texts. The analysis is based on data extracted from the *Penn Corpora of Historical English* (see http://www.ling.upenn.edu/hist-corpora/), that consists of 603 texts written between 1125 and 1900. 

Be start again by loading the data.


In [ ]:
pvd  <- base::readRDS(url("https://slcladal.github.io/data/pvd.rda", "rb")) %>%
  dplyr::mutate_if(is.character, factor) %>%
  # remove columns we do not need
  dplyr::select(-Date, -Genre, -Text)
# inspect
head(pvd); str(pvd)


Now, we implement the CIT and plot the results.



In [ ]:
# create initial conditional inference tree model
citd.ctree2 <- partykit::ctree(Prepositions ~ DateRedux + Region + GenreRedux,
                    data = pvd)
plot(citd.ctree2, gp = gpar(fontsize = 8)) # plot final ctree


We  can again save the figure using the `ggsave` command as shown below.



In [ ]:
ggsave("tree2.png")



## Prettifying your CIT tree

The easiest and most common way to visualize CITs is to simply use the `plot` function from `base R`. However, using this function does not allow to adapt and customize the visualization except for some very basic parameters. The `ggparty` function allows to use the `ggplot` syntax to customize CITs which allows more adjustments and is more aesthetically pleasing.

To generate this customized CIT, we activate the `ggparty` package and extract the significant p-values from the CIT object. We then plot the CIT and define the nodes, edges, and text elements as shown below. 


In [ ]:
# extract p-values
pvals <- unlist(nodeapply(citd.ctree, ids = nodeids(citd.ctree), function(n) info_node(n)$p.value))
pvals <- pvals[pvals <.05]
# plotting
ggparty(citd.ctree) +
  geom_edge() +
  geom_edge_label() +
  geom_node_label(line_list = list(aes(label = splitvar),
                                   aes(label = paste0("N=", nodesize, ", p", 
                                                      ifelse(pvals < .001, "<.001", paste0("=", round(pvals, 3)))), 
                                       size = 10)),
                  line_gpar = list(list(size = 13), 
                                   list(size = 10)), 
                  ids = "inner") +
  geom_node_label(aes(label = paste0("Node ", id, ", N = ", nodesize)),
    ids = "terminal", nudge_y = -0.0, nudge_x = 0.01) +
  geom_node_plot(gglist = list(
    geom_bar(aes(x = "", fill = LikeUser),
             position = position_fill(), color = "black"),
      theme_minimal(),
      scale_fill_manual(values = c("gray50", "gray80"), guide = "none"),
      scale_y_continuous(breaks = c(0, 1)),
    xlab(""), 
    ylab("Probability"),
    geom_text(aes(x = "", group = LikeUser,
                  label = stat(count)),
              stat = "count", position = position_fill(), vjust = 1.1)),
    shared_axis_labels = TRUE)


We can also use `position_dodge` (instead of `position_fill`) to display frequencies rather than probabilities as shown below. 



In [ ]:
# plotting
ggparty(citd.ctree) +
  geom_edge() +
  geom_edge_label() +
  geom_node_label(line_list = list(aes(label = splitvar),
                                   aes(label = paste0("N=", nodesize, ", p", 
                                                      ifelse(pvals < .001, "<.001", paste0("=", round(pvals, 3)))), 
                                       size = 10)),
                  line_gpar = list(list(size = 13), 
                                   list(size = 10)), 
                  ids = "inner") +
  geom_node_label(aes(label = paste0("Node ", id, ", N = ", nodesize)),
    ids = "terminal", nudge_y = 0.01, nudge_x = 0.01) +
  geom_node_plot(gglist = list(
    geom_bar(aes(x = "", fill = LikeUser),
             position = position_dodge(), color = "black"),
      theme_minimal(),
    theme(panel.grid.major = element_blank(), 
            panel.grid.minor = element_blank()),
      scale_fill_manual(values = c("gray50", "gray80"), guide = "none"),
      scale_y_continuous(breaks = seq(0, 100, 20),
                         limits = c(0, 100)),
    xlab(""), 
    ylab("Frequency"),
      geom_text(aes(x = "", group = LikeUser,
                    label = stat(count)),
                stat = "count", 
                position = position_dodge(0.9), vjust = -0.7)),
    shared_axis_labels = TRUE)


## Problems of Conditional Inference Trees

Like other tree-based methods, CITs are very intuitive, multivariate, non-parametric, they do not require large data sets, and they are easy to implement. Despite these obvious advantages, they have at least one major short coming compared to other, more sophisticated tree-structure models (in addition to the general issues that tree-structure models exhibit as discussed above: they are prone to **overfitting** which means that they fit the observed data very well but preform much worse when being applied to new data.

An extension which remedies this problem is to use a so-called ensemble method which grows many varied trees. The most common ensemble method is called a *Random Forest Analysis* but we cannot deal with them. If you want to learn more about random forests, have a look at [this LADAL tutorial on tree-based methods](https://ladal.edu.au/tree.html) and their implementation in R. 

***

[Back to the GitHub repo](https://github.com/MartinSchweinberger/TreesUBonn)

***
